# Cardiac Sub-Structure Segmentation Example

> Important: When running on Google Colab, ensure you change your runtime to use a GPU before
proceeding (Runtime->Change runtime type, select GPU as Hardware accelerator and press Save)

## Import Modules

In [1]:
try:
    import platipy
except:
    # We install platipy with the 'cardiac' extra since that contains some extra libraries we need.
    !pip install platipy[cardiac]
    import platipy

from matplotlib import pyplot as plt

import SimpleITK as sitk

from platipy.imaging.tests.data import get_lung_nifti
from platipy.imaging.projects.cardiac.run import run_hybrid_segmentation
from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com

## Download Test Data

This will download lung cancer patient CT scans, with contours of several structures.
This only has to be done once - if it is ran again don't worry, the same data will not be downloaded again!

In [2]:
data_path = get_lung_nifti()

## Load Test Image

Read in the image we want to automatically segment


In [3]:
test_pat_path = data_path.joinpath("LCTSC-Test-S1-201")
test_image = sitk.ReadImage(str(test_pat_path.joinpath("IMAGES/LCTSC_TEST_S1_201_0_CT_0.nii.gz")))

## Run Auto-segmentation

This will take some time, and will print updates along the way


In [4]:
pip install numpy==1.24.4 torch==2.1.2


In [5]:
auto_structures, _ = run_hybrid_segmentation(test_image)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

/root/.platipy/nnUNet_models/nnUNet/3d_lowres/Task400_OPEN_HEART_1FOLD
This model expects 1 input modalities for each image
Found 1 unique case ids, here are some examples: ['Task400_OPEN_HEART_1FOLD']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
emptying cuda cache
loading parameters for folds, all
using the following model files:  ['/root/.platipy/nnUNet_models/nnUNet/3d_lowres/Task400_OPEN_HEART_1FOLD/nnUNetTrainerV2__nnUNetPlansv2.1/all/model_final_checkpoint.model']
starting preprocessing generator
starting prediction...
preprocessing /tmp/tmpma470qt3/Task400_OPEN_HEA

## Save segmentations
Optionally write the automatic segmentations to disk


In [6]:
output_directory = test_pat_path.joinpath("substructures")
output_directory.mkdir(exist_ok=True)

for struct_name in list(auto_structures.keys()):
    sitk.WriteImage(auto_structures[struct_name], str(output_directory.joinpath(f"{struct_name}.nii.gz")))

print(f"Segmentations saved to: {output_directory}")

Segmentations saved to: data/nifti/lung/LCTSC-Test-S1-201/substructures


## Visualise Segmentations

Next, we can generate a nice figure to check what the segmentation looks like using platipy's ImageVisualiser

In [7]:
vis = ImageVisualiser(test_image, cut=get_com(auto_structures["Heart"]))
vis.add_contour({struct: auto_structures[struct] for struct in auto_structures.keys()})
vis.set_limits_from_label(auto_structures["Heart"], expansion=20)
vis.show()
snapshot_path = output_directory.joinpath(f"snapshot.png")
plt.savefig(snapshot_path)
print(f"Snapshot saved to: {snapshot_path}")

Snapshot saved to: data/nifti/lung/LCTSC-Test-S1-201/substructures/snapshot.png
